In [ ]:
! mkdir squad
! wget https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json -O squad/train-v2.0.json
! wget https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json -O squad/dev-v2.0.json

--2021-03-22 10:10:35--  https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 42123633 (40M) [application/json]
Saving to: ‘squad/train-v2.0.json’

squad/train-v2.0.js 100%[===================>]  40.17M   245MB/s    in 0.2s    

2021-03-22 10:10:36 (245 MB/s) - ‘squad/train-v2.0.json’ saved [42123633/42123633]

--2021-03-22 10:10:36--  https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4370528 (4.2M) [application/json]
Saving to: ‘squad/dev-v2.0.json’

squ

In [ ]:
import json
from pathlib import Path
 
def read_squad(path):
    path = Path(path)
    with open(path,'rb') as f:
        squad_dict = json.load(f)
 
    contexts = []
    questions = []
    answers = []
 
    for group in squad_dict["data"]:
        for passage in group["paragraphs"]:
            context = passage["context"]
            for qa in passage["qas"]:
                question = qa["question"]
                for answer in qa["answers"]:
                    contexts.append(context)
                    questions.append(question)
                    answers.append(answer)
    
    return contexts, questions, answers

In [ ]:
train_contexts, train_questions, train_answers = read_squad("squad/train-v2.0.json")
val_contexts, val_questions, val_answers = read_squad("squad/dev-v2.0.json")

Contexts and questions are string. The answers are dicts containing the subsequence of the passage with the correct answer as well as the integer indicating the character at which the answer begins. In order to train the model on this data we need

(1) the tokenized context/question pairs, and 

(2) Integers indicating at which token positions the answer begins and ends.


First, let's get the character position at which the answer ends in the passage

In [ ]:
def add_end_idx(answers, contexts):
    for answer, context in zip(answers, contexts):
        gold_text = answer["text"]
        start_idx = answer["answer_start"]
        end_idx = start_idx + len(gold_text)

        # sometimes squad answers are off by a character or two - fix this
        if context[start_idx:end_idx] == gold_text:
            answer['answer_end'] = end_idx
        elif context[start_idx-1:end_idx-1] == gold_text:
            answer["answer_start"] = start_idx + 1
            answer["answer_end"] = end_idx + 1  # When the gold label is off by one character
        elif context[start_idx-2:end_idx-2] == gold_text:
            answer["answer_start"] = start_idx + 2
            answer["answer_end"] = end_idx + 2  # When the gold label is off by two characters

add_end_idx(train_answers, train_contexts)
add_end_idx(val_answers, val_contexts)

In [ ]:
! pip install transformers

     |████████████████████████████████| 2.0MB 18.3MB/s 
     |████████████████████████████████| 3.2MB 55.9MB/s 
     |████████████████████████████████| 890kB 56.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=f9ed65cb3f02f036b9bcbd1340f2998c129871f372223fa1d6770afb6e4b7b9f
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
from transformers import DistilBertTokenizerFast

tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

train_encodings = tokenizer(train_contexts, train_questions, truncation=True, padding = True)
val_encodings = tokenizer(val_contexts, val_questions, truncation=True, padding=True)

In [ ]:
for count, i in enumerate(train_encodings):
    if count<5:
        print(i)

input_ids
attention_mask


In [ ]:
def add_token_positions(encodings, answers):
    start_positions = []
    end_positions = []
    for i in range(len(answers)):
        start_positions.append(encodings.char_to_token(i, answers[i]["answer_start"]))
        end_positions.append(encodings.char_to_token(i, answers[i]["answer_end"] - 1))

        # if start position is None, the answer passage has been truncated 
        if start_positions[-1] is None:
            start_positions[-1] = tokenizer.model_max_length
        
        if end_positions[-1] is None:
            end_positions[-1] = tokenizer.model_max_length

    encodings.update({"start_positions": start_positions, "end_positions": end_positions})


In [ ]:
add_token_positions(train_encodings, train_answers)
add_token_positions(val_encodings, val_answers)

In [ ]:
import torch

class SquadDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
        self.start_positions = encodings["start_positions"]
        self.end_positions = encodings["end_positions"]

    def __getitem__(self, idx):
        item = {key:torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['start_positions'] = torch.tensor(self.start_positions[idx])
        item["end_positions"] = torch.tensor(self.end_positions[idx]) 
        return item

    def __len__(self):
        return len(self.encodings.input_ids)

In [ ]:
for count, i in enumerate(train_encodings):
    if count<5:
        print(i)

input_ids
attention_mask
start_positions
end_positions


In [ ]:
train_dataset = SquadDataset(train_encodings)
val_dataset = SquadDataset(val_encodings)

In [ ]:
from transformers import DistilBertForQuestionAnswering
model = DistilBertForQuestionAnswering.from_pretrained("distilbert-base-uncased")

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForQuestionAnswering: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this mode

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
model = model.to(device)

In [ ]:
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)

# model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset             # evaluation dataset
)

trainer.train()

Step,Training Loss
10,6.041700
20,6.003300
30,5.990900
40,5.912100
50,5.808500
60,5.654600
70,5.437300
80,5.163400
90,4.965200
100,4.727700


TrainOutput(global_step=16281, training_loss=1.0046048394351323, metrics={'train_runtime': 13048.1198, 'train_samples_per_second': 1.248, 'total_flos': 5.310098044580045e+16, 'epoch': 3.0, 'init_mem_cpu_alloc_delta': 3174128, 'init_mem_gpu_alloc_delta': 0, 'init_mem_cpu_peaked_delta': 706932, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 1217183, 'train_mem_gpu_alloc_delta': 530958848, 'train_mem_cpu_peaked_delta': 98012047, 'train_mem_gpu_peaked_delta': 6540899328})

In [ ]:
trainer.evaluate()

{'epoch': 3.0,
 'eval_loss': 1.3620326519012451,
 'eval_mem_cpu_alloc_delta': 71945,
 'eval_mem_cpu_peaked_delta': 187107,
 'eval_mem_gpu_alloc_delta': 0,
 'eval_mem_gpu_peaked_delta': 2315895808,
 'eval_runtime': 380.0819,
 'eval_samples_per_second': 53.415}

In [ ]:
test_context = """
Machine learning (ML) is the study of computer algorithms that improve automatically through experience. It is seen as a part of artificial intelligence. Machine learning algorithms build a model based on sample data, known as "training data", in order to make predictions or decisions without being explicitly programmed to do so. Machine learning algorithms are used in a wide variety of applications, such as email filtering and computer vision, where it is difficult or unfeasible to develop conventional algorithms to perform the needed tasks.
"""

In [ ]:
question = "What are machine learning models based on?"

In [ ]:
test_encoding = tokenizer(test_context, question, truncation=True, padding=True)

In [ ]:
input_ids = torch.tensor(test_encoding["input_ids"]).to(device)
attention_mask = torch.tensor(test_encoding["attention_mask"]).to(device)

In [ ]:
import numpy as np

def to_check_result(test_encoding):
    input_ids = torch.tensor(test_encoding["input_ids"]).to(device)
    attention_mask = torch.tensor(test_encoding["attention_mask"]).to(device)
    with torch.no_grad():
        outputs = model(input_ids.unsqueeze(0), attention_mask.unsqueeze(0))
    start_label_indices = np.argmax(outputs[0].to('cpu').numpy())
    end_label_indices = np.argmax(outputs[1].to('cpu').numpy())
    return start_label_indices, end_label_indices

In [ ]:
print(start_label_indices)
print(end_label_indices)

36
37


In [ ]:
tokens = tokenizer.convert_ids_to_tokens(input_ids)
result =  " ".join(tokens[start_label_indices:end_label_indices+1]) 

In [ ]:
print(question)
print(f"Answer : {result}")

What are machine learning models based on?
Answer : sample data


In [ ]:
context = """
 Modern football originated in Britain in the 19th century. Though “folk football” had been played since medieval times with varying rules, the game began to be standardized when it was taken up as a winter game at public schools. Leo Messi is god of football. Football, also called association football or soccer, game in which two teams of 11 players, using any part of their bodies except their hands and arms, try to maneuver the ball into the opposing team’s goal. Only the goalkeeper is permitted to handle the ball and may do so only within the penalty area surrounding the goal. The team that scores more goals wins.
"""

In [ ]:
question_1 = "Which country started football"
question_2 = "Who is the god of football"

In [ ]:
test_encoding_1 = tokenizer(context, question_1, truncation=True, padding=True)
test_encoding_2 = tokenizer(context, question_2, truncation=True, padding=True)

In [ ]:
start_1, end_1 = to_check_result(test_encoding_1)
start_2, end_2 = to_check_result(test_encoding_2)

In [ ]:
tokens_1 = tokenizer.convert_ids_to_tokens(test_encoding_1["input_ids"])
tokens_2 = tokenizer.convert_ids_to_tokens(test_encoding_2["input_ids"])
result_1 =  " ".join(tokens_1[start_1:end_1 + 1])
result_2 =  " ".join(tokens_2[start_2:end_2 + 1]) 

In [ ]:
print(question_1)
print(f"Answer : {result_1}")

Which country started football
Answer : britain
Who is the god of football
leo mess ##i


In [ ]:
print(question_2)
print(f"Answer : {result_2}")

Who is the god of football
Answer : leo mess ##i
